In [18]:
import json
import tweepy
import time
import config_twit
import jsonpickle
import pandas as pd
import numpy as np

In [7]:
timestr = time.strftime("%Y%m%d-%H%M%S")
query = '#kathua OR "kathua" AND "rape"'
max_tweets = 2000

In [8]:
# Enter your keys/secrets as strings in the following fields. config_twit is a .py file that contains 
# my twitter api auth codes 

API_KEY = config_twit.api_key
API_SECRET = config_twit.api_secret
ACCESS_TOKEN = config_twit.ACCESS_TOKEN
ACCESS_TOKEN_SECRET = config_twit.ACCESS_TOKEN_SECRET

# assuming twitter_authentication.py contains each of the 4 oauth elements (1 per line)

auth = tweepy.OAuthHandler(API_KEY, API_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth)


In [9]:
# extract data from tweets that match search terms 

searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]
tweet_id = [s.id for s in searched_tweets]
user_id = [s.user.id for s in searched_tweets]
screen_name = [s.user.screen_name for s in searched_tweets]
followers = [s.user.followers_count for s in searched_tweets]
retweet_count = [s.retweet_count for s in searched_tweets]
locations = [s.user.location for s in searched_tweets]


%time
node:
{"name":"Mme.Burgon","group":7}

link:
{"source":1,"target":0,"value":1}


In [11]:
# df to parse data into network graph - here make df for convenience but later could have lists

screen_name = [s.user.screen_name for s in searched_tweets]
df = pd.DataFrame({'tweet_id':tweet_id, 'screen_name':screen_name, 'source':user_id, 'followers':followers, 'locations':locations})
most_followed_tweeters = df.sort_values(by='followers', ascending=False).head(10).drop_duplicates(subset='screen_name')
most_followed_tweets = zip(most_followed_tweeters.tweet_id, most_followed_tweeters.source, 
                           most_followed_tweeters.followers)


> Links list

In [12]:
# make links list: get retweet list of each tweet

api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True, compression=True)

users_retweeted_dict ={}

retweet_lst_names=[]
for status_id, user_id, followers in most_followed_tweets:
    
    retweet_lst=[]
    retweet_lst_name =[]
    
    for retweet_id in api.retweets(status_id):
        retweet_lst.append(retweet_id.user.id)
        retweet_lst_name.append(retweet_id.user.name)

    users_retweeted_dict[user_id] = retweet_lst
    retweet_lst_names.append(retweet_lst_name)

# flatten list
retweet_lst_names = [j for i in retweet_lst_names for j in i]

In [15]:
# then replace tweet id with index number

links_lst=[]
for k in users_retweeted_dict:

    if users_retweeted_dict[k]:
        for u in users_retweeted_dict[k]:
            links_lst.append({"source":k,"target":u})


In [19]:
# get followers counts

top_tweeters_follower_counts=[]
mft = most_followed_tweeters.followers

normalized_followers = np.round((np.abs(np.array(mft)-np.array(mft).mean())/mft.std())*10, 0)



count=0

for i in links_lst:
    for j,k in zip(most_followed_tweeters.source, normalized_followers ) :
        
        if i['source'] == j:
            
            top_tweeters_follower_counts.append(int(k))
    
    
# for k,v in zip(most_followed_tweeters.user_id, most_followed_tweeters.followers):
    
    
#    most_followed_tweeters.followers
# links_lst[0]['source']


In [20]:
# put all elements in link list together

count=0
count1=0
links_r =[]
for k,v in users_retweeted_dict.items():
    count1+=1
    for y,t in zip(most_followed_tweeters.source, normalized_followers ):
        if y == k:
            value = t
    for i in v:
        count+=1
        links_r.append({"source":count1, "target": count, "value": int(value)})
        

>Node list

In [21]:
# node: make "group" from location

location_india =[]
for i in most_followed_tweeters.locations:
    if 'India' or 'New Delhi' in i:
        location_india.append(1)
    else:
        location_india.append(0)
# 

In [22]:
# make nodes lst

nodes_lst = []
for x,y in zip(most_followed_tweeters.screen_name, location_india):
     nodes_lst.append({"name":x, "group":y})

for x,y in enumerate(retweet_lst_names):
     nodes_lst.append({"name":y, "group":2})



> export

In [23]:
# putting it together
data = {}
data['nodes'] = nodes_lst
data['links'] = links_r

In [24]:
import json
with open('data_twitter.json', 'w') as outfile:
    json.dump(data, outfile)